# FizzBuzz

FizzBuzz是一个简单的小游戏。游戏规则如下：从1开始往上数数，当遇到3的倍数的时候，说fizz，当遇到5的倍数，说buzz，当遇到15的倍数，就说fizzbuzz，其他情况下则正常数数。

我们可以写一个简单的小程序来决定要返回正常数值还是fizz, buzz 或者 fizzbuzz。

In [1]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if   i % 15 == 0: return 3
    elif i % 5  == 0: return 2
    elif i % 3  == 0: return 1
    else:             return 0
    
def fizz_buzz_decode(i, prediction):
    return [str(i), "fizz", "buzz", "fizzbuzz"][prediction]

print(fizz_buzz_decode(1, fizz_buzz_encode(1)))
print(fizz_buzz_decode(2, fizz_buzz_encode(2)))
print(fizz_buzz_decode(5, fizz_buzz_encode(5)))
print(fizz_buzz_decode(12, fizz_buzz_encode(12)))
print(fizz_buzz_decode(15, fizz_buzz_encode(15)))

1
2
buzz
fizz
fizzbuzz


我们首先定义模型的输入与输出(训练数据)

In [2]:
import numpy as np
import torch

NUM_DIGITS = 10

# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

trX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

然后我们用PyTorch定义模型

In [3]:
# Define the model
NUM_HIDDEN = 200
model = torch.nn.Sequential(
    torch.nn.Linear(NUM_DIGITS, NUM_HIDDEN),
    torch.nn.ReLU(),
    torch.nn.Linear(NUM_HIDDEN, 4)
)

- 为了让我们的模型学会FizzBuzz这个游戏，我们需要定义一个损失函数，和一个优化算法。
- 这个优化算法会不断优化（降低）损失函数，使得模型的在该任务上取得尽可能低的损失值。
- 损失值低往往表示我们的模型表现好，损失值高表示我们的模型表现差。
- 由于FizzBuzz游戏本质上是一个分类问题，我们选用Cross Entropyy Loss函数。
- 优化函数我们选用Stochastic Gradient Descent。

In [4]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

以下是模型的训练代码

In [5]:
# Start training it
BATCH_SIZE = 128
for epoch in range(10000):
    for start in range(0, len(trX), BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]

        y_pred = model(batchX)
        loss = loss_fn(y_pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Find loss on training data
    loss = loss_fn(model(trX), trY).item()
    print('Epoch:', epoch, 'Loss:', loss)

Epoch: 0 Loss: 1.1668494939804077
Epoch: 1 Loss: 1.1497050523757935
Epoch: 2 Loss: 1.1457303762435913
Epoch: 3 Loss: 1.144050121307373
Epoch: 4 Loss: 1.1430590152740479
Epoch: 5 Loss: 1.1423530578613281
Epoch: 6 Loss: 1.141793966293335
Epoch: 7 Loss: 1.141324520111084
Epoch: 8 Loss: 1.1409151554107666
Epoch: 9 Loss: 1.1405487060546875
Epoch: 10 Loss: 1.1402195692062378
Epoch: 11 Loss: 1.139917016029358
Epoch: 12 Loss: 1.1396394968032837
Epoch: 13 Loss: 1.139383316040039
Epoch: 14 Loss: 1.1391444206237793
Epoch: 15 Loss: 1.1389219760894775
Epoch: 16 Loss: 1.1387107372283936
Epoch: 17 Loss: 1.138510823249817
Epoch: 18 Loss: 1.138321876525879
Epoch: 19 Loss: 1.138140082359314
Epoch: 20 Loss: 1.1379668712615967
Epoch: 21 Loss: 1.1377989053726196
Epoch: 22 Loss: 1.1376368999481201
Epoch: 23 Loss: 1.1374794244766235
Epoch: 24 Loss: 1.1373268365859985
Epoch: 25 Loss: 1.13717782497406
Epoch: 26 Loss: 1.137033462524414
Epoch: 27 Loss: 1.1368935108184814
Epoch: 28 Loss: 1.136756181716919
Epoch: 

Epoch: 232 Loss: 1.105139136314392
Epoch: 233 Loss: 1.104842185974121
Epoch: 234 Loss: 1.1045632362365723
Epoch: 235 Loss: 1.104186773300171
Epoch: 236 Loss: 1.10389244556427
Epoch: 237 Loss: 1.1035687923431396
Epoch: 238 Loss: 1.1032609939575195
Epoch: 239 Loss: 1.1029456853866577
Epoch: 240 Loss: 1.102630376815796
Epoch: 241 Loss: 1.1023272275924683
Epoch: 242 Loss: 1.1019614934921265
Epoch: 243 Loss: 1.101680874824524
Epoch: 244 Loss: 1.101294994354248
Epoch: 245 Loss: 1.1009786128997803
Epoch: 246 Loss: 1.1006560325622559
Epoch: 247 Loss: 1.1002707481384277
Epoch: 248 Loss: 1.0998283624649048
Epoch: 249 Loss: 1.0995324850082397
Epoch: 250 Loss: 1.0992140769958496
Epoch: 251 Loss: 1.0987837314605713
Epoch: 252 Loss: 1.0985275506973267
Epoch: 253 Loss: 1.0981616973876953
Epoch: 254 Loss: 1.097812294960022
Epoch: 255 Loss: 1.0975083112716675
Epoch: 256 Loss: 1.0970970392227173
Epoch: 257 Loss: 1.096839427947998
Epoch: 258 Loss: 1.096476435661316
Epoch: 259 Loss: 1.0961607694625854
Epo

Epoch: 461 Loss: 0.9935463666915894
Epoch: 462 Loss: 0.9918325543403625
Epoch: 463 Loss: 0.9915032982826233
Epoch: 464 Loss: 0.9909509420394897
Epoch: 465 Loss: 0.9898850321769714
Epoch: 466 Loss: 0.9898425340652466
Epoch: 467 Loss: 0.9889059662818909
Epoch: 468 Loss: 0.988537609577179
Epoch: 469 Loss: 0.987535297870636
Epoch: 470 Loss: 0.9868772625923157
Epoch: 471 Loss: 0.9862043261528015
Epoch: 472 Loss: 0.9854514598846436
Epoch: 473 Loss: 0.9854786992073059
Epoch: 474 Loss: 0.9837821125984192
Epoch: 475 Loss: 0.9833390116691589
Epoch: 476 Loss: 0.982793927192688
Epoch: 477 Loss: 0.9821511507034302
Epoch: 478 Loss: 0.9823982119560242
Epoch: 479 Loss: 0.9804536700248718
Epoch: 480 Loss: 0.9809328317642212
Epoch: 481 Loss: 0.9792651534080505
Epoch: 482 Loss: 0.9786654114723206
Epoch: 483 Loss: 0.9780780673027039
Epoch: 484 Loss: 0.9788267016410828
Epoch: 485 Loss: 0.9768629670143127
Epoch: 486 Loss: 0.9766762852668762
Epoch: 487 Loss: 0.9763917326927185
Epoch: 488 Loss: 0.974860072135

Epoch: 689 Loss: 0.780883252620697
Epoch: 690 Loss: 0.7806301712989807
Epoch: 691 Loss: 0.7777056694030762
Epoch: 692 Loss: 0.7786710262298584
Epoch: 693 Loss: 0.7754403352737427
Epoch: 694 Loss: 0.7744646668434143
Epoch: 695 Loss: 0.7726650834083557
Epoch: 696 Loss: 0.7725747227668762
Epoch: 697 Loss: 0.7703855633735657
Epoch: 698 Loss: 0.7699649930000305
Epoch: 699 Loss: 0.7687216401100159
Epoch: 700 Loss: 0.7672252655029297
Epoch: 701 Loss: 0.7654209733009338
Epoch: 702 Loss: 0.7644813060760498
Epoch: 703 Loss: 0.7630064487457275
Epoch: 704 Loss: 0.7610107064247131
Epoch: 705 Loss: 0.7607892751693726
Epoch: 706 Loss: 0.7588924169540405
Epoch: 707 Loss: 0.7591493725776672
Epoch: 708 Loss: 0.7570282816886902
Epoch: 709 Loss: 0.7544611692428589
Epoch: 710 Loss: 0.7529799342155457
Epoch: 711 Loss: 0.7525683641433716
Epoch: 712 Loss: 0.7519304156303406
Epoch: 713 Loss: 0.7495789527893066
Epoch: 714 Loss: 0.7485259771347046
Epoch: 715 Loss: 0.7467026710510254
Epoch: 716 Loss: 0.7453794479

Epoch: 917 Loss: 0.48738226294517517
Epoch: 918 Loss: 0.4857498109340668
Epoch: 919 Loss: 0.4850448668003082
Epoch: 920 Loss: 0.4842173457145691
Epoch: 921 Loss: 0.48217856884002686
Epoch: 922 Loss: 0.48130545020103455
Epoch: 923 Loss: 0.48049649596214294
Epoch: 924 Loss: 0.4786425232887268
Epoch: 925 Loss: 0.47814133763313293
Epoch: 926 Loss: 0.47768306732177734
Epoch: 927 Loss: 0.47561973333358765
Epoch: 928 Loss: 0.4741375148296356
Epoch: 929 Loss: 0.47324642539024353
Epoch: 930 Loss: 0.47269243001937866
Epoch: 931 Loss: 0.47153159976005554
Epoch: 932 Loss: 0.4698362350463867
Epoch: 933 Loss: 0.46904727816581726
Epoch: 934 Loss: 0.4682150185108185
Epoch: 935 Loss: 0.4673254191875458
Epoch: 936 Loss: 0.4656442105770111
Epoch: 937 Loss: 0.46483340859413147
Epoch: 938 Loss: 0.46315547823905945
Epoch: 939 Loss: 0.4631907641887665
Epoch: 940 Loss: 0.4612119197845459
Epoch: 941 Loss: 0.460086852312088
Epoch: 942 Loss: 0.4594508707523346
Epoch: 943 Loss: 0.45780155062675476
Epoch: 944 Loss

...

Epoch: 8972 Loss: 0.005418440792709589
Epoch: 8973 Loss: 0.005417798180133104
Epoch: 8974 Loss: 0.00541679048910737
Epoch: 8975 Loss: 0.005416033789515495
Epoch: 8976 Loss: 0.005415129009634256
Epoch: 8977 Loss: 0.005414000246673822
Epoch: 8978 Loss: 0.005413432139903307
Epoch: 8979 Loss: 0.005411972291767597
Epoch: 8980 Loss: 0.005410990212112665
Epoch: 8981 Loss: 0.005410301964730024
Epoch: 8982 Loss: 0.005409949459135532
Epoch: 8983 Loss: 0.005408761091530323
Epoch: 8984 Loss: 0.005407758988440037
Epoch: 8985 Loss: 0.005406708922237158
Epoch: 8986 Loss: 0.005406033247709274
Epoch: 8987 Loss: 0.00540538365021348
Epoch: 8988 Loss: 0.0054040090180933475
Epoch: 8989 Loss: 0.005403263494372368
Epoch: 8990 Loss: 0.005402152426540852
Epoch: 8991 Loss: 0.005401428788900375
Epoch: 8992 Loss: 0.0054006632417440414
Epoch: 8993 Loss: 0.005399635061621666
Epoch: 8994 Loss: 0.005398732144385576
Epoch: 8995 Loss: 0.005397552624344826
Epoch: 8996 Loss: 0.005397094879299402
Epoch: 8997 Loss: 0.00539

Epoch: 9181 Loss: 0.005235978402197361
Epoch: 9182 Loss: 0.005234754178673029
Epoch: 9183 Loss: 0.005233618896454573
Epoch: 9184 Loss: 0.005232764407992363
Epoch: 9185 Loss: 0.005231623072177172
Epoch: 9186 Loss: 0.005231254268437624
Epoch: 9187 Loss: 0.005230266135185957
Epoch: 9188 Loss: 0.005229339934885502
Epoch: 9189 Loss: 0.005228797439485788
Epoch: 9190 Loss: 0.005227732937783003
Epoch: 9191 Loss: 0.005227032583206892
Epoch: 9192 Loss: 0.005226561333984137
Epoch: 9193 Loss: 0.005225161090493202
Epoch: 9194 Loss: 0.005224314983934164
Epoch: 9195 Loss: 0.005223381798714399
Epoch: 9196 Loss: 0.005222604144364595
Epoch: 9197 Loss: 0.005221843253821135
Epoch: 9198 Loss: 0.005221089348196983
Epoch: 9199 Loss: 0.005219846032559872
Epoch: 9200 Loss: 0.0052193705923855305
Epoch: 9201 Loss: 0.005218127742409706
Epoch: 9202 Loss: 0.005217535421252251
Epoch: 9203 Loss: 0.005216644145548344
Epoch: 9204 Loss: 0.005215587094426155
Epoch: 9205 Loss: 0.0052149356342852116
Epoch: 9206 Loss: 0.005

Epoch: 9390 Loss: 0.005063197109848261
Epoch: 9391 Loss: 0.005062356125563383
Epoch: 9392 Loss: 0.005061390809714794
Epoch: 9393 Loss: 0.005060589872300625
Epoch: 9394 Loss: 0.0050595952197909355
Epoch: 9395 Loss: 0.005059308838099241
Epoch: 9396 Loss: 0.005058226175606251
Epoch: 9397 Loss: 0.005057244095951319
Epoch: 9398 Loss: 0.005056485999375582
Epoch: 9399 Loss: 0.005056146997958422
Epoch: 9400 Loss: 0.005055214744061232
Epoch: 9401 Loss: 0.00505444360896945
Epoch: 9402 Loss: 0.005053277593106031
Epoch: 9403 Loss: 0.005052586551755667
Epoch: 9404 Loss: 0.005052253138273954
Epoch: 9405 Loss: 0.005051169544458389
Epoch: 9406 Loss: 0.005050230771303177
Epoch: 9407 Loss: 0.005049392115324736
Epoch: 9408 Loss: 0.005048668477684259
Epoch: 9409 Loss: 0.0050479271449148655
Epoch: 9410 Loss: 0.005047339014708996
Epoch: 9411 Loss: 0.005046178121119738
Epoch: 9412 Loss: 0.005045334808528423
Epoch: 9413 Loss: 0.00504468334838748
Epoch: 9414 Loss: 0.005043935030698776
Epoch: 9415 Loss: 0.00504

Epoch: 9599 Loss: 0.004900644067674875
Epoch: 9600 Loss: 0.004900023806840181
Epoch: 9601 Loss: 0.004899354185909033
Epoch: 9602 Loss: 0.004898751154541969
Epoch: 9603 Loss: 0.0048978617414832115
Epoch: 9604 Loss: 0.004896991886198521
Epoch: 9605 Loss: 0.004896241240203381
Epoch: 9606 Loss: 0.004895361606031656
Epoch: 9607 Loss: 0.004894856829196215
Epoch: 9608 Loss: 0.00489421421661973
Epoch: 9609 Loss: 0.004893091972917318
Epoch: 9610 Loss: 0.004892551805824041
Epoch: 9611 Loss: 0.0048916395753622055
Epoch: 9612 Loss: 0.004891189746558666
Epoch: 9613 Loss: 0.004890246782451868
Epoch: 9614 Loss: 0.004889336880296469
Epoch: 9615 Loss: 0.004888414870947599
Epoch: 9616 Loss: 0.004887972958385944
Epoch: 9617 Loss: 0.004886982496827841
Epoch: 9618 Loss: 0.004886488895863295
Epoch: 9619 Loss: 0.0048857079818844795
Epoch: 9620 Loss: 0.004885060712695122
Epoch: 9621 Loss: 0.004884514957666397
Epoch: 9622 Loss: 0.004883501213043928
Epoch: 9623 Loss: 0.004882719833403826
Epoch: 9624 Loss: 0.004

Epoch: 9808 Loss: 0.004747764207422733
Epoch: 9809 Loss: 0.004747104365378618
Epoch: 9810 Loss: 0.004746419843286276
Epoch: 9811 Loss: 0.004745925310999155
Epoch: 9812 Loss: 0.004745137877762318
Epoch: 9813 Loss: 0.0047442726790905
Epoch: 9814 Loss: 0.004743429832160473
Epoch: 9815 Loss: 0.004742722492665052
Epoch: 9816 Loss: 0.004742254503071308
Epoch: 9817 Loss: 0.004741345997899771
Epoch: 9818 Loss: 0.004740588832646608
Epoch: 9819 Loss: 0.004739824216812849
Epoch: 9820 Loss: 0.00473915645852685
Epoch: 9821 Loss: 0.004738375078886747
Epoch: 9822 Loss: 0.004737976472824812
Epoch: 9823 Loss: 0.004737155977636576
Epoch: 9824 Loss: 0.004736445844173431
Epoch: 9825 Loss: 0.004735919646918774
Epoch: 9826 Loss: 0.004735059570521116
Epoch: 9827 Loss: 0.0047343820333480835
Epoch: 9828 Loss: 0.0047336844727396965
Epoch: 9829 Loss: 0.004732683300971985
Epoch: 9830 Loss: 0.004732123576104641
Epoch: 9831 Loss: 0.004731439054012299
Epoch: 9832 Loss: 0.004730874206870794
Epoch: 9833 Loss: 0.004730

Epoch: 9999 Loss: 0.004615018609911203(H=100)
最后我们用训练好的模型尝试在1到100这些数字上玩FizzBuzz游戏

In [6]:
# Output now
testX = torch.Tensor([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
with torch.no_grad():
    testY = model(testX)
predictions = zip(range(1, 101), list(testY.max(1)[1].data.tolist()))

print([fizz_buzz_decode(i, x) for (i, x) in predictions])

['1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', '17', 'fizz', '19', 'buzz', 'fizz', '22', '23', 'fizz', '25', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', 'fizz', '34', 'buzz', 'fizz', '37', '38', 'fizz', 'buzz', '41', '42', '43', '44', 'fizzbuzz', '46', '47', 'fizz', '49', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', 'fizz', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', 'fizz', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', 'fizz', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


In [7]:
print(np.sum(testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])))
testY.max(1)[1].numpy() == np.array([fizz_buzz_encode(i) for i in range(1,101)])

98


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])